# Vorlesung 8.1
# Zeichenketten

---
 **Hinweis:**
 Diese interaktiven Webseiten beschreiben parallel zu den Vorlesungsfolien das jeweilige Stoffgebiet. Zellen mit C Quelltext können mittels der Tastenkombination \[Shift\] + \[Enter\] kompiliert und ausgeführt werden. Es wird Ihnen empfohlen, Änderungen in diversen Zellen vorzunehmen um ein Gefühl für die Sprache C zu entwickeln.
 
---

## Zeichenketten in C

In der Sprache C gibt es keinen eigenen Datentyp für Zeichenketten. Sie werden mit Feldern von Zeichen repräsentiert. Literale von Zeichenketten werden mit doppeltem Hochkomma definiert:
```C
"Hallo, Welt!"
```
Gespeichert werden Zeichenketten in Feldern des Typs `char`:
```C
char text[13] = "Hallo, Welt!";
```
Sie können auch mit `scanf` eingelesen und mit `printf` ausgegeben werden:

In [ ]:
/* Ausgabe von Zeichenketten */
/* Wir definieren eine lange Zeichenkette, damit der User beliebigen Text eingeben kann*/
char text[100] = ""; /* Wie immer, sollte jede Variable initialisiert werden. Hier eine leere Zeichenkette */

/* Text vom User eingeben lassen */
printf("Bitte geben Sie einen Text ein: ");
scanf("%s", text);

/* Nun geben wir den Inhalt von text aus */
printf("%s\n", text);

Im obigen Beispiel wird der Text korrekt ausgegeben. Aber woher weiß `printf`, wie lange der eingegebene Text war?

Zeichenketten werden in C immer mit dem Wert `0` beendet. In der ASCII Tabelle entspricht das dem Zeichen `\0`, dem sogenannten *Null Character*. Dies kann leicht überprüft werden, indem man von jedem Zeichen den Zahlenwert aus der ASCII Tabelle ausgibt:

In [ ]:
/* Ausgabe aller Zeichen einer Zeichenkette */
char text[20] = "Hallo, Welt!";

printf("Text | ASCII Werte\n");
{
    long i = 0;
    for(i = 0; i < 20; ++i) {
        printf("%-7c%d\n", text[i], text[i]);
    }
}
printf("printf: %s<--Ende der Zeichenkette", text);

Im obigen Beispiel sind einige Eigenschaften von Zeichenketten erkennbar:
* Wie in anderen Feldern, werden die nicht initialisierten Werte auf `0` gesetzt.
* `printf` beendet die Ausgabe mit dem ersten `\0` Zeichen, das es findet.

Dadurch, dass nach der eigentlichen Zeichenkette immer noch ein `\0` geschrieben werden muss, wird eine extra Stelle benötigt. Wie bei anderen Feldern, kann die Länge automatisch vom Compiler ermittelt werden:

In [ ]:
/* Initialisierung mit Zeichenketten */
char text[] = "Hallo"; /* Hallo hat fünf Buchstaben */
char text1[] = {'H', 'a', 'l', 'l', 'o', '\0'}; /* Die übliche Schreibweise für Felder */

/* Hallo\0 benötigt aber 6 Zeichen */
printf("%ld\n", sizeof(text));
printf("%s\n", text1);

In [ ]:
/* Nach der Initialisierung kann die Zeichenkette
    nicht mehr verwendet werden um text zu setzen.
    text ist ein Zeiger und kann nicht verändert werden! (Siehe Vorlesung 6.1)
    Hier müsste jedes Zeichen einzeln gesetzt werden. */
char text[6];
text = "Hallo";
printf("%s\n", text);

## Zeiger auf Zeichenketten

Wenn Felder mit Zeichenketten initialisiert werden, wird jedes Zeichen in das Feld kopiert und kann dort benutzt werden:

<img src="images/stringinit.png" width=400>

**ACHTUNG**: Jede Zeichenkette ist ein Feld. Der Bezeichner des Feldes kann auch einem Zeiger zugewiesen werden, weil er ein konstanter Zeiger auf das Feld ist. Allerdings kann das zu Problemen führen, wenn der Zeiger nicht konstant ist (Bezeichner von Feldern sind immer konstante Zeiger). Das heißt, der Quelltext:
```C
char *ptr = "Hallo";
```
gibt zwar keine Fehlermeldung, sollte aber **auf jeden Fall vermieden** werden! Wenn `ptr` verändert wird, gibt es keine Möglichkeit mehr auf die Zeichenkette zuzugreifen, weil die Speicheradresse verloren ist. 

Außerdem liegt die konstante Zeichenkette `"Hallo"` in einem geschützten Speicherbereich, auf den nicht geschrieben werden darf. Wenn man nun versucht an diese Adresse zu schreiben:
```
ptr[0] = 'c';
```
beendet das Programm mit einem Speicherzugriffsfehler, engl. *Segmentation Fault* (siehe Vorlesung 6.1).

**Richtig wäre**:
```C
char text[] = "Hallo";
```

Im Speicher sind Felder für Zeichenkette und Zeiger auf Zeichenketten sehr unterschiedlich:
<img src="images/stringptr.png" width=500>

## Elementare Funktionen für Zeichenketten

Wenn eine Zeichenkette nicht bei der Initialisierung gesetzt wird, muss jedes Zeichen einzeln gesetzt werden, was mühsam sein kann. Um die Arbeit mit Zeichenketten zu erleichtern, bietet die Standardbibliothek einige Funktionen, die im Header `string.h` deklariert sind:

| Funktion | Beschreibung |
| -------- | ------------ |
| `strcat(s, t)` | hängt `t` an `s` an |
| `strncat(s, t, n)` | hängt die ersten `n` Zeichen von `t` an `s` an |
| `strcmp(s, t)` | vergleicht `s` und `t` und gibt zurück:<br>**0**: `s` und `t` sind gleich<br>**&lt;0**: das erste ungleiche Zeichen ist in `s` kleiner<br>**&gt;0**: das erste ungleiche Zeichen ist in `s` größer |
| `strncmp(s, t, n)` | wie `strcmp(s, t)` aber nur für die ersten `n` Zeichen |
| `strcpy(s, t)` | kopiert `t` nach `s` |
| `strncpy(s, t, n)` | kopiert die ersten `n` zeichen von `t` nach `s` |
| `strlen(s)` | gibt die Länge von `s` ohne Nullcharakter zurück |

**Vorsicht**: `strcat` steht für *string concatenate*, also das Zusammenführen von Zeichenketten und nicht für eine Katze, die in `string.h` lebt:


                                     ,
              ,-.       _,---._ __  / \
             /  )    .-'       `./ /   \
            (  (   ,'            `/    /|
             \  `-"             \'\   / |
              `.              ,  \ \ /  |      ___
               /`.          ,'-`----Y   |     /   \
              (            ;        |   '    |\    |   <-- Ball of strings
              |  ,-.    ,-' ASCII   |  /      \|__/    
              |  | (   |    Chars   | /         \    ___
              )  |  \  `.___________|/           \__/
              `--'   `--'

In [ ]:
/* Einige Funktionen aus string.h */
#include <stdio.h>
#include <string.h>

int main() {
    /* Zwei strings sind gegeben und werden in einen kopiert */
    char vorname[] = "Emmett";
    char nachname[] = "Brown";
    char name[30];
    
    strcpy(name, vorname); /* vorname wird nach name kopiert */
    strcat(name, " "); /* ein Leerzeichen an name anhängen */
    strcat(name, nachname); /* nachname an name anhängen */
    
    /* Nun kann der gesamte Name als eine Zeichenkette ausgegeben werden */
    printf("Zitat von %s:\n", name);

    
    /* Mittels strcmp kann er nun mit einem ganzen Namen verglichen werden */
    if(strcmp(name, "Arthur Dent") == 0) {
        printf("Gibt es denn Tee auf diesem Raumschiff?");
    } else if(strcmp(name, "Emmett Brown") == 0) {
        printf("Wo wir hinfahren, brauchen wir keine Strassen!");
    } else {
        printf("Der Name ist nicht bekannt.");
    }
        
    return 0;
}

In [ ]:
/* Teil einer Zeichenkette extrahieren */
#include <stdio.h>
#include <string.h>

int main() {
    /* Die Funktionsdeklaration von printf als Text */
    char text[] = "int printf(const char*, ...)";
    char parameter[100]; /* willkuerliche Laenge, könnte auch kuerzer/laenger sein */
    long length = 0;
    
    /* Wir wollen die Funktionsparameter extrahieren.
        Ein Zeiger auf den Anfang der Klammer,
        einen an das Ende. (siehe weiter Unten zum Suchen in Zeichenketten) */
    char *klammer1 = strchr(text, '(');
    char *klammer2 = strchr(text, ')');
    
    /* Wir wollen die Klammer allerdings nicht dabei.
        Zeiger um ein Element verschieben */
    klammer1 = klammer1 + 1; /* zeigt jetzt auf c */
    klammer2 = klammer2 - 1; /* zeigt jetzt auf . */
    
    /* Wie viele Zeichen brauchen wir.
        Die Anzahl der Elemente zwischen ( und )*/
    length = klammer2 - klammer1 + 1;
    
    /* Jetzt den Teil zwischen den Klammern nach parameter kopieren */
    strncpy(parameter, klammer1, length);
    /* Weil \0 noch nicht dabei war, muss es von Hand gesetzt werden */
    parameter[length] = '\0';
    
    printf("Die Parameter sind:\n");
    printf("%s\n", parameter);
        
    return 0;
}

## Formatierte Zeichenketten

`printf` steht für *printf formatted*, weil das erste Argument ein *format string* ist, also eine Zeichenkette, die das Format der Ausgabe bestimmt. Diese Format-Zeichenkette kann auch benutzt werden, um eine andere Zeichenkette zu schreiben. Hierfür gibt es die Funktion `sprintf` in `stdio.h`, wo auch `printf` definiert ist.

Allerdings kann `sprintf` sehr lange Zeichenketten schreiben und da im Vorhinein nicht klar ist, wie lange die tatsächliche Zeichenkette wird, sollte immer nur `snprintf` verwendet werden, welches maximal n Zeichen schreibt.

In [ ]:
/* In eine Zeichenkette schreiben */
#define LENGTH 20

char text[LENGTH];
long answer = 42;

/* text ist das Feld in das geschrieben werden soll,
    LENGTH ist die maximale Anzahl an Zeichen(inklusive \0 am Ende)
    Es können alle Platzhalter und Variablen wie in printf verwendet werden. */
snprintf(text, LENGTH, "i = %ld\n", answer);
printf("%s\n", text);

Analog zu `sprintf` gibt es auch eine Funktion zum Einlesen von Zeichenketten: `sscanf`. Nachdem ihr eine gültige Zeichenkette übergeben werden muss, wird die Länge nicht zusätzlich benötigt weil das Ende der Zeichenkette sowieso mit einem `'\0'` markiert sein muss.

Genau wie `scanf` gibt sie die Anzahl der konvertierten Elemente zurück:

In [ ]:
long answer = 0;
char text[20] = "42"; /* Eine Zeichenkette mit den Zeichen: '4', '2', '\0' */

long anzahl = sscanf(text, "%ld", &answer);

/* Jetzt kann die Eingabe überprüft werden */
if(anzahl != 1) {
    printf("Fehler!\n");
} else {
    long i = 0;
    for(i = 0; i < answer; ++i) {
        printf("-");
    }
}

## Suchen in Zeichenketten

Der Header `string.h` ist ebenfalls sehr nützlich für das Durchsuchen von Zeichenketten. Es werden mehrere Funktionen bereitgestellt, die das Suchen nach Zeichen oder Zeichenketten in Zeichenketten erleichtern:

| Funktion | Beschreibung |
| -------- | ------------ |
| `strchr(s, c)` | sucht das Zeichen `c` in der Zeichenkette `s` |
| `strrchr(s, c)` | wie `strchr(s, c)` aber rückwärts |
| `strstr(s, t)` | sucht die Zeichenkette `t` in der Zeichenkette `s` |
| `strrstr(s, t)` | wie `strstr(s, t)` aber rückwärts |

### Rückgabewert
Die obigen Funktionen geben alle einen Zeiger auf das erste gefundene Zeichen zurück. Wenn nach einer Zeichenkette gesucht wurde, ist der Rückgabewert ein Zeiger auf das erste Zeichen der gefundenen Zeichenkette.

Wenn die Suche erfolglos war, wird ein `NULL` Zeiger zurückgegeben, also ein Zeiger auf die Speicheradresse `0`.

In [ ]:
/* Jedes Leerzeichen in einen Zeilenumbruch ändern */
#include <stdio.h>
#include <string.h>

int main() {
    char text[] = "Das Pferd frisst keinen Gurkensalat.";
    
    /* Zeiger auf ein Zeichen in text. */
    char *pointer = &text[0]; /* Anfangs auf das erste Element */
    
    /* Zeichenkette bevor einer Änderung */
    printf("%s\n", text);
    
    /* Suche nach einem Leerzeichen in text.
        Wenn die Suche 0 zurück gibt, wurde keines mehr gefunden */
    while((pointer = strchr(pointer, ' ')) != 0) {
        /* Das Leerzeichen durch \n ersetzen */
        *pointer = '\n';
    }
    
    printf("%s\n", text);
    
    return 0;
}

## Iterieren mit Zeigern

Zeiger auf Zeichenketten können auch benutzt werden, um einmal über jedes Element zu gehen. Zum Beispiel mittels einer `for` Schleife:

In [ ]:
/* Verschlüsselung nach Julius Caesar */
#include <stdio.h>
#include <string.h>

int main() {
    char text[] = "Die spinnen, die Roemer.";
    char *pointer = 0; /* Variablen immer initialisieren */
    const char key = 7;
    
    printf("Original: %s\n", text);
    
    /* Über das ganze Feld iterieren, bis zum Null Character */
    for(pointer = &text[0]; *pointer != '\0'; ++pointer) {
        *pointer = (*pointer - 'A' + key) % 58 + 'A';
    }
    
    printf("Verschlüsselt: %s\n", text);
    
    /* Wieder über das Feld iterieren zum entschlüsseln */
    for(pointer = &text[0]; *pointer != '\0'; ++pointer) {
        *pointer = (*pointer - 'A' - key) % 58 + 'A'; 
    }
    
    printf("Entschlüsselt: %s\n", text);
    
    return 0;
}